# Create an image classification pipeline

This notebook was written in Jupyter notebooks on the Vertex AI Workbench. You may need to adjust the code to run it in other contexts (e.g. Colab). The code here is the pipeline version of the model created in "image-classification.ipynb."

**Important:** Using Vertex AI can run up significant costs. Be sure to estimate your resource usage before running these notebooks.

You can find the dataset used in this model on [Kaggle.com](https://www.kaggle.com/abhinavnayak/catsvdogs-transformed).


## Install libraries, import libraries, set constants

In [ ]:
! pip install google-cloud-aiplatform==1.3.0 kfp  google-cloud-pipeline-components

In [24]:
import kfp
from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]

print(PROJECT_ID)

project_id = PROJECT_ID
pipeline_root_path = "gs://2022-03-01-aggie-demo/pipeline_root"

aggie-data-science-demo


## Define the pipeline

In [31]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(name="automl-image-training")
def pipeline(project_id: str = PROJECT_ID):
    ds_op = gcc_aip.ImageDatasetCreateOp(
        project=project_id,
        display_name="pipeline-cat-or-dog-ds",
        gcs_source="gs://2022-03-01-aggie-demo/training_data.jsonl",
        import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
    )

    training_job_run_op = gcc_aip.AutoMLImageTrainingJobRunOp(
        project=project_id,
        display_name="pipeline-cat-or-dog-pipeline",
        prediction_type="classification",
        model_type="CLOUD",
        dataset=ds_op.outputs["dataset"],
        model_display_name="pipeline-cat-or-dog-model",
        budget_milli_node_hours=8000,
        multi_label=False,
    )

    create_endpoint_op = gcc_aip.EndpointCreateOp(
        project=project_id,
        display_name = "pipeline-cat-or-dog-endpoint",
    )

    model_deploy_op = gcc_aip.ModelDeployOp(
        model=training_job_run_op.outputs["model"],
        endpoint=create_endpoint_op.outputs['endpoint'],
        automatic_resources_min_replica_count=1,
        automatic_resources_max_replica_count=1,
    )

In [32]:
from kfp.v2 import compiler  # noqa: F811
compiler.Compiler().compile(pipeline_func=pipeline, package_path='image_classification.json')

In [ ]:
from kfp.v2.google.client import AIPlatformClient
api_client = AIPlatformClient(project_id=PROJECT_ID, region="us-central1")
response = api_client.create_run_from_job_spec(
    "image_classification.json",
    pipeline_root=pipeline_root_path,
    parameter_values={"project_id": PROJECT_ID},
)